In [1]:
import pandas as pd
import numpy as np
import os

In [5]:
x_test_select = pd.read_csv('x_test_select.csv', index_col = 'LigandID')
x_test_select

,LR-L2_AtomPairs2D,LR-L2_AtomPairs2DCount,LR-L2_CDK,LR-L2_CDKextended,LR-L2_CDKgraphonly,LR-L2_EState,LR-L2_KlekotaRoth,LR-L2_KlekotaRothCount,LR-L2_MACCS,LR-L2_PubChem,...,kNN_CDK,kNN_CDKextended,kNN_CDKgraphonly,kNN_EState,kNN_KlekotaRoth,kNN_KlekotaRothCount,kNN_MACCS,kNN_PubChem,kNN_Substructure,kNN_SubstructureCount
LigandID,,,,,,,,,,,,,,,,,,,,,
CHEMBL3971505,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CHEMBL4114320,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
CHEMBL4106855,1,1,1,0,1,1,1,1,1,1,...,0,0,0,0,1,1,1,0,1,1
CHEMBL3654689,1,1,0,0,0,0,1,1,0,1,...,1,0,0,0,0,1,1,0,0,0
CHEMBL4107477,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [6]:
name = 'Stack'
fp = 'All'
from joblib import load
Model = load('models/'+name+fp+'.joblib')
y_test_predict  = Model.predict(x_test_select)
y_test_predict  = pd.DataFrame(y_test_predict, index=x_test_select.index, columns=['StackMAPK'])
y_test_predict

/home/tarapong/anaconda3/envs/mapk-1.2.2/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


,StackMAPK
LigandID,
CHEMBL3971505,1
CHEMBL4114320,1
CHEMBL4106855,1
CHEMBL3654689,0
CHEMBL4107477,0


In [15]:
#AD
x_train = pd.read_csv('Stack_All_train.csv', index_col = 'LigandID')
#Compute Applicability Domain
def nearest_neighbor_AD_predict(x_test, k, z=0.5):
    from sklearn.neighbors import NearestNeighbors
    import numpy as np
    import pandas as pd
    nn = NearestNeighbors(n_neighbors = k, algorithm='brute', metric='euclidean').fit(x_train)
    distance, index = nn.kneighbors(x_train)
    #calculate mean and sd of distance in train set
    di = np.mean(distance, axis = 1)
    dk = round(np.mean(di),2)
    sk = round(np.std(di),2)
    distance, index = nn.kneighbors(x_test)
    di = np.mean(distance, axis = 1)
    AD_status = []
    for i in range(len(di)):
        if di[i] < dk + (z * sk):
            AD_status.append('within_AD')
        else:
            AD_status.append('outside_AD')

    # Create DataFrame with index from x_test and the respective status
    df = pd.DataFrame(AD_status,  index=x_test.index, columns=['AD_status'])
    return df
test_ad = nearest_neighbor_AD_predict(x_test_select, 4, z=0.5)

#Combine
y_test_predict = pd.concat([y_test_predict, test_ad], axis=1)
y_test_predict


,StackMAPK,AD_status
LigandID,,
CHEMBL3971505,1,within_AD
CHEMBL4114320,1,within_AD
CHEMBL4106855,1,outside_AD
CHEMBL3654689,0,outside_AD
CHEMBL4107477,0,outside_AD
